In [26]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
file_path = 'incidents.csv'  # Update with the correct file path if necessary
df = pd.read_csv(file_path)

# Data cleaning: Convert 'Funds Lost' to numeric and 'Date' to datetime
df['Funds Lost'] = df['Funds Lost'].str.replace(',', '').astype(float)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Generate four‑month intervals for each year to create period labels
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Quarter'] = (df['Month'] - 1) // 4
df['Year-Month'] = (
    df['Year'].astype(str) + '-'
    + df['Quarter'].map({0: '01', 1: '05', 2: '09'})
)
df = df.sort_values(by=['Year', 'Quarter'])
df['Year-Month-Full'] = df['Date'].dt.strftime('%Y-%m')

# Ensure the dataset has a "Custody" column; if not, default all rows to "Non-Custodial"
if 'Custody' not in df.columns:
    df['Custody'] = "Non-Custodial"

# Separate the data into four‑month intervals and intermediate months
mask = df['Month'].isin([1, 5, 9])
four_df = df[mask].drop_duplicates(subset=['Year-Month'], keep='first').copy()
inter_df = df[~mask].copy()

# Generate a full range of period labels from min to max year
all_periods = pd.date_range(
    start=f"{df['Year'].min()}-01",
    end=f"{df['Year'].max()}-12",
    freq='4M'
).strftime('%Y-%m').tolist()
base = pd.DataFrame({'Year-Month': all_periods})

# Merge & sort (Code A approach)
full_df = (
    base
    .merge(four_df, on='Year-Month', how='left')
    .fillna({'Funds Lost': 0, 'Custody': "Non-Custodial"})
    .sort_values(by='Funds Lost', ascending=False)
)
inter_df = inter_df.sort_values(by='Funds Lost', ascending=False)

# Sizeref calculation (bubbles sized in M USD)
max_f = df['Funds Lost'].max() / 1e6
sizeref = 2. * max_f / (90**2)

# Colours
cust_col = "#008080"
noncust_col = "#FEBE7E"
colors_full = [cust_col if c == "Custodial" else noncust_col for c in full_df['Custody']]
colors_int  = [cust_col if c == "Custodial" else noncust_col for c in inter_df['Custody']]

# Build figure
fig = go.Figure()
y0 = -0.05

# Four‑month bubbles
fig.add_trace(go.Scatter(
    x=full_df['Year-Month'], y=[y0]*len(full_df),
    mode='markers',
    marker=dict(
        size=full_df['Funds Lost'] / 1e6,
        sizemode='area', sizeref=sizeref, sizemin=5,
        color=colors_full
    ),
    text=[
        f"Date: {ym}, Funds Lost: ${fund:.2f}M"
        for ym, fund in zip(full_df['Year-Month'], full_df['Funds Lost'] / 1e6)
    ],
    showlegend=False
))

# Intermediate bubbles
fig.add_trace(go.Scatter(
    x=inter_df['Year-Month-Full'], y=[y0]*len(inter_df),
    mode='markers',
    marker=dict(
        size=inter_df['Funds Lost'] / 1e6,
        sizemode='area', sizeref=sizeref, sizemin=5,
        color=colors_int
    ),
    text=[
        f"Date: {ym}, Funds Lost: ${fund:.2f}M"
        for ym, fund in zip(inter_df['Year-Month-Full'], inter_df['Funds Lost'] / 1e6)
    ],
    showlegend=False
))

# Custom bubble-size legend
legend_x = ['2026-08', '2027-05', '2028-03', '2029-04']
legend_sz = [50, 100, 200, 500]
legend_lbl= ['50', '100', '200', '500']

fig.add_trace(go.Scatter(
    x=legend_x, y=[y0]*len(legend_x),
    mode='markers+text',
    marker=dict(
        size=legend_sz, sizemode='area', sizeref=sizeref, sizemin=5,
        color="#D3D3D3"
#         4d4d4d
    ),
    text=legend_lbl,
    textposition='middle center',
    textfont=dict(size=20, color='black'),
    showlegend=False
))

# "Amount Lost" annotation
fig.add_annotation(
    x='2028-02', y=0.025, yref='paper',
    text="Amount Lost (M USD)",
    showarrow=False,
    font=dict(size=20, color='black')
)

# ——— Custody labels with square icons to the right of text ———
label_y = -0.355

# Custodial
fig.add_trace(go.Scatter(
    x=["2029-04"], y=[0.045],
    mode='text+markers',
    text=["Custodial"],
    textposition='middle left',      # text on left, marker on right
    textfont=dict(size=16, color=cust_col),
    marker=dict(symbol='square', size=12, color=cust_col),
    showlegend=False
))

# Non‑Custodial
fig.add_trace(go.Scatter(
    x=["2029-04"], y=[0.015],
    mode='text+markers',
    text=["Non Custodial"],
    textposition='middle left',
    textfont=dict(size=16, color=noncust_col),
    marker=dict(symbol='square', size=12, color=noncust_col),
    showlegend=False
))
# ————————————————————————————————————————————————————————————

# Layout
fig.update_layout(
    xaxis=dict(
        title='Date (Year-Month)',
        tickmode='array', tickvals=all_periods, ticktext=all_periods,
        tickangle=-90, tickfont=dict(size=16, color='black'),
        titlefont=dict(size=20, color='black')
    ),
    yaxis=dict(
        title='Loss (M USD)', visible=True, showticklabels=False,
        titlefont=dict(size=20, color='black'), title_standoff=0
    ),
    paper_bgcolor='white', plot_bgcolor='white',
    height=250, width=1100,
    margin=dict(l=0, r=0, t=0, b=0)
)

# Save & show
fig.write_image("Timeline_of_Wallet_Hacks.pdf", format='pdf', engine='kaleido', scale=8)

fig.show()


